# FPI Model

### This model compares ESPN FPI (Football Power Index) to Bovada odds and suggests bets

### Organization
1. Get up to date FPI from ESPN- DONE
2. Get up to date odds from Bovada - difficult, use espn chalk instead
3. Money line implied odds comparison
4. Point spread implied odds comparison

_______________________________________________________________________________________________________________________________

In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
# up to date FPI function
# returns pandas dataframe with current FPI table

def getFPI():
    url = 'https://www.espn.com/nfl/fpi'
    r = requests.get(url)
    html = r.text
    
    # ESPN splits the FPI table into two sides
    
    # put the left table (team names) into a pandas dataframe    
    soup = BeautifulSoup(html)
    table1 = soup.find('table', {"class": "Table Table--align-right Table--fixed Table--fixed-left"})
    rows = table1.find_all('tr')
    teams_data = []
    for row in rows[2:]:
        cols = row.find_all('td')
        cols = [element.text.strip() for element in cols]
        teams_data.append([element for element in cols if element])   

    teams_df = pd.DataFrame(teams_data)
    
    # put the right side table (FPI and other stats) into a pandas dataframe
    table2 = soup.find('table', {"class": "Table Table--align-right"})
    rows = table2.find_all('tr')
    stats_data = []
    for row in rows[2:]:
        cols = row.find_all('td')
        cols = [element.text.strip() for element in cols]
        stats_data.append([element for element in cols if element])   

    stats_df = pd.DataFrame(stats_data)
    
    # combine into one dataframe and update headings
    df = pd.merge(teams_df, stats_df, left_index=True, right_index=True)
    headers = {'0_x': 'TEAM', 
               '0_y': 'W-L', 
               1 : 'FPI', 
               2: 'RK', 
               3: 'TRND', 
               4: 'OFF', 
               5: 'DEF', 
               6: 'ST', 
               7: 'SOS', 
               8: 'REM_SOS', 
               9: 'AVG_WP'}
    df = df.rename(index=str,columns=headers)
    return df

In [3]:
def getESPNlines():
    url = 'https://www.espn.com/nfl/lines'
    dfs = pd.read_html(url)
    
    return dfs

In [4]:
dfs_in = getESPNlines()

In [5]:
line_dfs = dfs_in[:]
for i, df in enumerate(dfs_in):

    line_dfs[i] = df

## How to calculate implied odds (%) based on American Money Line
 
### Negative American Odds (favorite)
implied probability = negative odds / (negative odds + 100) * 100

### Positive American Odds (underdog)
implied probability = 100 / (positive odds + 100) * 100

In [6]:
# add implied odds column to dfs and compare
lines_list = line_dfs[:]
for i, df in enumerate(line_dfs):
    # sort: underdogs, favorites    
    df = df.sort_values('LINE', ascending=False)
    
    # add total and points for each team row
    total, spread = df.iloc[0]['LINE'], df.iloc[1]['LINE']
    df['TOTAL'] = [total, total]
    df['SPREAD'] = [-spread, spread]
    
    # drop unnecessary columns
    df = df.drop(['REC (ATS)', 'OPEN', 'LINE'], axis=1)
  
    # sort: underdog, favorite
    df = df.sort_values(['ML'], ascending=False)
    
    # calculate implied odds from ML
    for index, row in df.iterrows():
        if row['ML'] > 0:
            underdog = (100*100/(row['ML']+100))
        else:
            favorite = 100*abs(row['ML'])/(abs(row['ML'])+100)
    df['Implied_Odds'] = [underdog,favorite]
    
    # calculate ML edge (FPI - Implied Odds)
    df['FPI'] = df['FPI'].str.strip(' %')
    df['FPI'] = df['FPI'].astype(float)
    
    df['ML_Edge'] = df['FPI'] - df['Implied_Odds']
    df['U/F?'] = ['underdog', 'favorite']
    
    df.columns.values[0] = 'TEAM'
    
    lines_list[i] = df

In [7]:
# convert list of games to dataframe, organize columns, index by team
lines = pd.concat(lines_list)
lines = lines[['TEAM', 'ML', 'SPREAD', 'TOTAL', 'FPI', 'Implied_Odds', 'ML_Edge', 'U/F?']]
team_lines = lines.set_index('TEAM')

points_FPI = getFPI()
points_FPI = points_FPI.set_index('TEAM')
points_FPI['FPI'] = points_FPI['FPI'].astype(float)
points_FPI

,W-L,FPI,RK,TRND,OFF,DEF,ST,SOS,REM_SOS,AVG_WP
TEAM,,,,,,,,,,
Kansas City Chiefs,4-1,8.7,1,--,8.7,0.2,-0.2,14,24,10
Baltimore Ravens,4-1,7.3,2,--,2.9,4.1,0.4,25,30,2
Seattle Seahawks,5-0,5.3,3,2,5.4,-0.1,0.0,16,14,8
Los Angeles Rams,4-1,5.2,4,4,3.5,1.7,-0.1,26,5,6
Green Bay Packers,4-0,4.9,5,1,6.8,-1.5,-0.4,20,18,1
Tampa Bay Buccaneers,3-2,4.7,6,2,0.9,4.3,-0.5,22,11,11
New Orleans Saints,3-2,3.7,7,--,4.5,-1.1,0.3,12,12,18
San Francisco 49ers,2-3,3.1,8,5,0.9,1.9,0.3,31,1,14
New England Patriots,2-2,2.3,9,1,-1.8,3.8,0.3,3,15,15


# NOTE

ESPN SOS (strength of schedule) and AVG_WP (average in-game win probability) appear to be inversely related (intuitive), but discrepencies could lead to betting edge. Room for more analysis

### Money Line Bets Sorted by Best Edge
only displays ML bets with a positive edge

In [8]:
# take positive edge, round values, and organize
pos_edge = team_lines[team_lines.ML_Edge > 0]
pos_edge = pos_edge[['ML', 'FPI', 'Implied_Odds', 'ML_Edge', 'U/F?']]
pos_edge = pos_edge.sort_values('ML_Edge', ascending=False)
pos_edge.round(decimals=3)

,ML,FPI,Implied_Odds,ML_Edge,U/F?
TEAM,,,,,
Tampa Bay Buccaneers,-105,53.0,22.727,30.273,underdog
Denver Broncos,340,32.5,22.727,9.773,underdog
Washington,130,52.7,43.478,9.222,underdog
Cincinnati Bengals,310,32.2,24.390,7.810,underdog
San Francisco 49ers,150,46.6,40.000,6.600,underdog
Cleveland Browns,170,42.7,37.037,5.663,underdog
Philadelphia Eagles,360,26.3,21.739,4.561,underdog
New York Jets,340,26.6,22.727,3.873,underdog
Atlanta Falcons,180,36.9,35.714,1.186,underdog


### Point Spreads
ESPN FPI is defined as the expected point margin vs an average opponent
Assume an average opponent has an FPI of zero

FPI favorite expected point spread = Favorite FPI - Underdog FPI
FPI underdog expected point spread = -FPI favorite expected point spread

In [9]:
# loop over each matchup and calculate
spread_list = lines_list.copy()
for i, df in enumerate(lines_list):
    
    # drop unnecessary columns
    df = df.drop(['ML', 'FPI', 'TOTAL', 'Implied_Odds', 'ML_Edge'], axis=1)

    # index to teams
    df = df.set_index('TEAM')

    # add FPI, calculate FPI spread, calculate points edge
    df = pd.concat([df, points_FPI['FPI']], axis=1, join='inner')
    fpi_spread = df.iloc[0]['FPI'] - df.iloc[1]['FPI']
    df['FPI_spread'] = [-fpi_spread, fpi_spread]
    df['Points_Edge'] = df['SPREAD'] - df['FPI_spread']
     
    spread_list[i] = df

In [10]:
# convert list to a dataframe, organize, take positive edge
spreads = pd.concat(spread_list)
spreads = spreads[['SPREAD', 'FPI', 'FPI_spread', 'Points_Edge', 'U/F?']]

points_edge = spreads[spreads.Points_Edge > 0]
points_edge = points_edge.sort_values('Points_Edge', ascending=False)
points_edge

,SPREAD,FPI,FPI_spread,Points_Edge,U/F?
TEAM,,,,,
Washington,3.0,-5.2,-2.1,5.1,underdog
Denver Broncos,9.0,-2.3,4.6,4.4,underdog
New York Jets,9.5,-7.9,6.9,2.6,underdog
Cleveland Browns,3.5,0.7,1.3,2.2,underdog
Cincinnati Bengals,7.5,-4.7,5.4,2.1,underdog
Arizona Cardinals,-1.5,-0.1,-3.2,1.7,favorite
Kansas City Chiefs,-5.0,8.7,-6.7,1.7,favorite
Chicago Bears,1.5,-1.4,0.5,1.0,underdog
San Francisco 49ers,3.0,3.1,2.1,0.9,underdog


__________________________________________________________

# Final Results

## Money Line Edge

In [11]:
pos_edge

,ML,FPI,Implied_Odds,ML_Edge,U/F?
TEAM,,,,,
Tampa Bay Buccaneers,-105,53.0,22.727273,30.272727,underdog
Denver Broncos,340,32.5,22.727273,9.772727,underdog
Washington,130,52.7,43.478261,9.221739,underdog
Cincinnati Bengals,310,32.2,24.390244,7.809756,underdog
San Francisco 49ers,150,46.6,40.000000,6.600000,underdog
Cleveland Browns,170,42.7,37.037037,5.662963,underdog
Philadelphia Eagles,360,26.3,21.739130,4.560870,underdog
New York Jets,340,26.6,22.727273,3.872727,underdog
Atlanta Falcons,180,36.9,35.714286,1.185714,underdog


## Points Edge

In [12]:
points_edge

,SPREAD,FPI,FPI_spread,Points_Edge,U/F?
TEAM,,,,,
Washington,3.0,-5.2,-2.1,5.1,underdog
Denver Broncos,9.0,-2.3,4.6,4.4,underdog
New York Jets,9.5,-7.9,6.9,2.6,underdog
Cleveland Browns,3.5,0.7,1.3,2.2,underdog
Cincinnati Bengals,7.5,-4.7,5.4,2.1,underdog
Arizona Cardinals,-1.5,-0.1,-3.2,1.7,favorite
Kansas City Chiefs,-5.0,8.7,-6.7,1.7,favorite
Chicago Bears,1.5,-1.4,0.5,1.0,underdog
San Francisco 49ers,3.0,3.1,2.1,0.9,underdog


___________________________________________________________

# Stuff to Add

* add opponent, home/away, record, weather to dataframes
* add strength of schedule to date anaylsis